# Data Preparation

With a better understanding of data representation, let's now turn to preparing data for input into a machine learning pipeline. In the case of unsupervised learning, a simple matrix-level representation can suffice for input to machine learning models; we also need accompanying labels.

Often, traffic capture datasets are accompanied by labels. These labels can tell us something about the accompanying data points (i.e., flows, packets) in the traffic, and can be used to train the model for future prediction.

Automated tools exist for assigning labels to traffic flows, including [pcapML](https://nprint.github.io/pcapml/). Before we use those tools, we will do some automatic preparation and labeling from an existing dataset, a log4j trace from [malware traffic analysis](https://www.malware-traffic-analysis.net/2021/12/20/index.html) and a regular trace.

You can use the NetML traffic library to generate features.

In [3]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)

from netml.pparser.parser import PCAP
from netml.utils.tool import dump_data, load_data

import pandas as pd

## Load the Packet Capture Files

Load the Log4j and HTTP packet capture files and extract features from the flows. You can feel free to compute features manually, although it will likely be more convenient at this point to use the `netML` library.

In [6]:
pcap = PCAP('data/log4j.pcap', flow_ptks_thres=2, verbose=10)
hcap = PCAP('data/http.pcap', flow_ptks_thres=2, verbose=10)


## Convert the Packet Capture Into Flows

Find the function in `netml` that converts the pcap file into flows. Examing the resulting data structure. What does it contain?

In [7]:
pcap.pcap2flows()

'_pcap2flows()' starts at 2023-10-18 15:28:01
pcap_file: data/log4j.pcap
ith_packets: 0
ith_packets: 10000
ith_packets: 20000
ith_packets: 30000
ith_packets: 40000
ith_packets: 50000
ith_packets: 60000
ith_packets: 70000
ith_packets: 80000
len(flows): 44707
total number of flows: 44707. Num of flows < 2 pkts: 39869, and >=2 pkts: 4838 without timeout splitting.
kept flows: 4766. Each of them has at least 2 pkts after timeout splitting.
flow_durations.shape: (4766, 1)
          col_0
count  4766.000
mean     12.068
std     191.592
min       0.000
25%       0.454
50%       5.075
75%       7.065
max   12889.480
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4766 entries, 0 to 4765
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   col_0   4766 non-null   float64
dtypes: float64(1)
memory usage: 37.4 KB
None
0th_flow: len(pkts): 6
1000th_flow: len(pkts): 5
2000th_flow: len(pkts): 5
3000th_flow: len(pkts): 6
4000th_flow: len(pkt

In [8]:
hcap.pcap2flows()

'_pcap2flows()' starts at 2023-10-18 15:28:29
pcap_file: data/http.pcap
ith_packets: 0
ith_packets: 10000
ith_packets: 20000
len(flows): 593
total number of flows: 593. Num of flows < 2 pkts: 300, and >=2 pkts: 293 without timeout splitting.
kept flows: 293. Each of them has at least 2 pkts after timeout splitting.
flow_durations.shape: (293, 1)
        col_0
count 293.000
mean   11.629
std    15.820
min     0.000
25%     0.076
50%     0.455
75%    20.097
max    46.235
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   col_0   293 non-null    float64
dtypes: float64(1)
memory usage: 2.4 KB
None
0th_flow: len(pkts): 4
After splitting flows, the number of subflows: 291 and each of them has at least 2 packets.
'_pcap2flows()' ends at 2023-10-18 15:28:41 and takes 0.1897 mins.


## Explore the Flows

How many flows are in each of these pcaps? (Use the `netml` library output to determine the size of each data structure.)

In [11]:
print(len(pcap.flows))
print(len(hcap.flows))

4795
291


## Normalize the Shapes of Each Feature Set

If you loaded the two pcaps with `netml` separately, the features will not be of the same dimension.  

1. Adjust your data frames so that the two have the same number of columns.
2. Merge (i.e., concatenate) the two data frames, but preserve the labels as a separate vector called "target".

In [82]:
pcap.flow2features('IAT', fft=False, header=False)
hcap.flow2features('IAT', fft=False, header=False)

'_flow2features()' starts at 2023-10-18 15:48:09
True
'_flow2features()' ends at 2023-10-18 15:48:09 and takes 0.0016 mins.
'_flow2features()' starts at 2023-10-18 15:48:09
True
'_flow2features()' ends at 2023-10-18 15:48:09 and takes 0.0007 mins.


In [89]:
p = pd.DataFrame(pcap.features)
h = pd.DataFrame(hcap.features)
print(p.shape)
print(h.shape)
p['label'] = 1
h['label'] = -1
all = h.merge(p, how='outer')

(4795, 5)
(291, 92)


In [90]:
print(all.shape)
all.head(10)

(5086, 93)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,label
0,0.037,30.003,0.037,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1
1,0.020,30.020,0.020,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1
2,15.494,4.999,15.131,5.336,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1
3,15.433,5.057,15.072,5.399,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1
4,0.058,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1
5,0.012,0.001,0.030,0.000,0.042,0.016,0.000,0.048,0.032,0.001,0.012,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-1
6,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000

## Try Your Data on a Model

You should now have data that can be input into a model with scikit-learn. Import the scikit-learn package (`sklearn`) and a classification model of your choice to test that you can train your model with the above data. 

Hint: The function you want to call is `fit`.

**Note:** If you plan to use a linear model such as logistic regression, your label should be a numerical value, and if the problem is a binary classification model, as in this case, then the appropriate label should be 0 and 1 for each respective class. (If you are using a tree-based model, then the labels could take any format.)

(Note that we have not done anything here except train the model with all of the data. To evaluate the model, we will need to split the data into train and test sets.)

In [91]:
from sklearn.linear_model import LogisticRegression

In [98]:
features = all.loc[:,:4]
targets = all['label']

lr = LogisticRegression(random_state=0).fit(features, targets)

## Test Your Trained Model

We used the entire dataset to train the model in this example (no split), and so of course the model will be well-fit to all of the data. To simply test that your trained model works, call `predict` using a feature vector that you generate by hand (e.g., from scratch, using a random set of numbers, from another pcap).

## Bonus  

Consider the following extensions to the above exercise:

* Concatenate or combine multiple features (either from `netml` or some of your own) into the same feature representation.
* Normalize your features so that they are in the same range (helpful for some models).

The above exercise gives you an example of how to generate features from a packet capture, attach labels to the dataset, and train a model using the labeled data. 

## Looking Ahead

Many other steps exist in the machine learning pipeline, including splitting the data into training and test sets, tuning model parameters, and evaluating the model. These will be the next steps we walk through.